# script to pull bike counts

- concat SDOT, Bellevue and WSDOT bike counts into `observed_bike_counts_2023.csv`

## SDOT API

April 2023 weekday bike counts from 10 [SDOT bike trails](https://www.seattle.gov/transportation/projects-and-programs/programs/bike-program/bike-counters)

- find API endpoints on [Open Data Portal](https://data.seattle.gov/)
- 2023 data is missing in Elliott Bay Trail in Myrtle Edwards Park. Used 2022 data instead.


In [88]:
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# this csv file contains bike counter locations and their corresponding API keys, column names and last updated date
bike_api_source = pd.read_csv("sdot_bike_counter_api_source.csv", dtype=str)
# connect to the SDOT Socrata API
client = Socrata("data.seattle.gov", None)

In [ ]:
# get the API key from SDOT Socrata
def get_count(api_key, d_1, d_2, year):
    results = client.get(api_key,
        select="date_trunc_ymd(date) as day, date_extract_dow(date) as dow, sum(" + d_1 + ") as direct_1_count, sum(" + d_2 + ") as direct_2_count",
        where="date >= '" + year + "-04-01' and date <= '" + year + "-04-30' and dow in (1, 2, 3, 4, 5)",
        group="day, dow", 
        order="day",
        limit=2000)
    results_df = pd.DataFrame.from_records(results)
    return results_df

# test the function with a sample API key and counter names
# get_count("65db-xm6k", "fremont_bridge_nb", "fremont_bridge_sb", "2023")
# get_count("4qej-qvrz", "bike_north", "bike_south", "2023")

,day,dow,direct_1_count,direct_2_count
0,2023-04-03T00:00:00.000,1,671,1046
1,2023-04-04T00:00:00.000,2,1178,1909
2,2023-04-05T00:00:00.000,3,1004,1806
3,2023-04-06T00:00:00.000,4,461,862
4,2023-04-07T00:00:00.000,5,769,1208
5,2023-04-10T00:00:00.000,1,409,643
6,2023-04-11T00:00:00.000,2,759,1256
7,2023-04-12T00:00:00.000,3,1118,1776
8,2023-04-13T00:00:00.000,4,772,1267
9,2023-04-14T00:00:00.000,5,1101,1483


In [90]:
daily_bike_count = pd.DataFrame()

for _, row in bike_api_source.iterrows():
    # read information from the bike_api_source.csv
    bridge = row['Location']
    # print(bridge)
    api_key = row['api_key']
    data_year = row['last_year']
    direct_1 = None
    direct_2 = None
    bound = None
    if not pd.isna(row['NB']):
        direct_1, direct_2 = row['NB'], row['SB']
        bound = "NB/SB"
    else:
        direct_1, direct_2 = row['EB'], row['WB']
        bound = "EB/WB"
    # print(f"Bridge: {bridge}, API key: {api_key}, Direct 1: {direct_1}, Direct 2: {direct_2}, Direct 2: {bound}, Year: {data_year}")

    # get the bike count data from API
    df_count = get_count(api_key, direct_1, direct_2, data_year)
    # add location info
    df_count['Location'] = bridge
    df_count['data_year'] = data_year
    df_count['bound'] = bound
    df_count[['direct_1_count', 'direct_2_count']] = df_count[['direct_1_count', 'direct_2_count']].apply(pd.to_numeric)
    df_count['total_count'] = df_count['direct_1_count'] + df_count['direct_2_count']

    daily_bike_count = pd.concat([daily_bike_count, df_count], ignore_index=True)
    

In [95]:
# aggregate to weekday counts in each location
df_final = daily_bike_count.groupby(['Location', 'data_year', 'bound'])[['direct_1_count', 'direct_2_count', 'total_count']].\
    apply(lambda x: x.mean().round(0)).reset_index()
df_final['measure'] = 'weekday'
df_final['month'] = 'April'

df_final = df_final[['Location', 'data_year', 'month', 'measure', 'bound', 'direct_1_count', 'direct_2_count', 'total_count']].copy()
df_final

,Location,data_year,month,measure,bound,direct_1_count,direct_2_count,total_count
0,2nd Ave Cycle Track North of Marion St,2023,April,weekday,NB/SB,292.0,351.0,643.0
1,39th Ave NE Greenway at NE 62nd St,2018,April,weekday,NB/SB,101.0,28.0,129.0
2,Broadway Cycle Track North Of E Union St,2021,April,weekday,NB/SB,109.0,125.0,234.0
3,Burke Gilman Trail north of NE 70th St,2023,April,weekday,NB/SB,335.0,345.0,679.0
4,Chief Sealth Trail North of Thistle,2021,April,weekday,NB/SB,32.0,25.0,57.0
5,Elliott Bay Trail in Myrtle Edwards Park,2022,April,weekday,NB/SB,357.0,366.0,723.0
6,Fremont Bridge,2023,April,weekday,NB/SB,890.0,1460.0,2350.0
7,MTS Trail west of I-90 Bridge,2021,April,weekday,EB/WB,254.0,110.0,364.0
8,NW 58th St Greenway at 22nd Ave NW,2022,April,weekday,EB/WB,44.0,34.0,78.0
9,Spokane St,2023,April,weekday,EB/WB,277.0,303.0,580.0


In [ ]:
# save data to csv
df_final.to_csv("R:/e2projects_two/2023_base_year/bike_count/sdot_observed_bike_counts_2023.csv", index=False)

## Bellevue Bike counters

- requested by email

In [ ]:
bellevue_2024 = pd.read_excel("R:/e2projects_two/2023_base_year/bike_count/Jan-Dec_2024.xlsx", sheet_name="Data_2024")

all_cols = ['108th Ave SE SB',
       '108th Ave SE NB', '108th Ave SE', '112th Ave NE NB', '112th Ave NE SB',
       '112th Ave NE ', '114th Ave SE SB', '114th Ave SE NB', '114th Ave SE ',
       '116th Ave NE NB ', '116th Ave NE SB', '116th Ave NE',
       '118th Ave SE SB', '118th Ave SE NB', '118th Ave SE', '140th Ave SE SB',
       '140th Ave SE NB', '140th Ave SE', '145th Pl SE SB', '145th Pl SE NB',
       '145th Pl SE ', '164th Ave NE ', '164th Ave NE SB', '164th Ave NE NB',
       'Lake Hills Blvd EB', 'Lake Hills Blvd WB', 'Lake Hills Blvd',
       'Main St EB', 'Main St WB', 'Main St', 'NE 12th St IN',
       'NE 12th St OUT', 'NE 12th St', 'Newport Way IN', 'Newport Way OUT',
       'Newport Way', 'Richards Rd NB', 'Richards Rd SB ', 'Richards Rd',
       'W Lake Sammamish Pkwy SE SB', 'W Lake Sammamish Pkwy SE NB',
       'W Lake Sammamish Pkwy SE', 'I-90 Trail Ped', 'I-90 Trail Bike',
       'SR 520 Trail Ped EB', 'SR 520 Trail Ped WB', 'SR 520 Trail Ped',
       'SR 520 Trail Bike EB', 'SR 520 Trail Bike WB', 'SR 520 Trail Bike',
       'Eastrail Ped SB', 'Eastrail Ped NB', 'Eastrail Ped',
       'Eastrail Bike SB', 'Eastrail Bike NB', 'Eastrail Bike']

In [114]:
# filter to April
bellevue_Apr_2024 = bellevue_2024[(bellevue_2024['Time']>="2024-04-01") & (bellevue_2024['Time']<="2024-04-30") &\
                                  (bellevue_2024['Weekday']<6)].copy()
# Convert the date to datetime64
bellevue_Apr_2024['date'] = bellevue_2024['Time'].dt.date

# daily count
daily_bike_count_bellevue = bellevue_Apr_2024.groupby(['date','Year', 'Month', 'Weekday'])[all_cols].sum().reset_index()

In [121]:
avg = daily_bike_count_bellevue.groupby(['Year'])[all_cols].mean().round(0).reset_index()

In [122]:
df_final_bellevue = avg.melt(id_vars=["Year"], var_name="Location", value_name="count")

- manual formmatting from here to match SDOT data

In [ ]:
df_final_bellevue.to_csv("R:/e2projects_two/2023_base_year/bike_count/bellevue_bike_apr2024.csv", index=False)


## WSDOT bike counts

- downloaded from WSDOT website: https://ftp.wsdot.wa.gov/gis/GeoDataDistribution/Maps/noscale/DOT_TDO/BikePTR/
- WSDOT locations: https://wsdot.wa.gov/data/tools/bikepedcounts/

In [125]:
wsdot_2023 = pd.read_csv("R:/e2projects_two/2023_base_year/bike_count/wsdot_PTRBikePedSummary2023.csv")

C:\Users\Modeller\AppData\Local\Temp\ipykernel_33084\4015321179.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  wsdot_2023 = pd.read_csv("R:/e2projects_two/2023_base_year/bike_count/wsdot_PTRBikePedSummary2023.csv")


In [137]:
df_wsdot_2023 = wsdot_2023.loc[
    (wsdot_2023['StartIntervalDateTime']>=20230401000000) &\
        (wsdot_2023['StartIntervalDateTime']<=20230430240000)].copy()

df_wsdot_2023['date'] = df_wsdot_2023['StartIntervalDateTime'].astype(str).str.slice(0, 8)
df_wsdot_2023 = df_wsdot_2023.groupby(['LocationDescription', 'Longitude', 'Latitude', 'date', 'DirectionOfTravel'])['BicyclistCount'].sum().reset_index()

df_wsdot_2023 = df_wsdot_2023.groupby(['LocationDescription', 'Longitude', 'Latitude', 'DirectionOfTravel'])['BicyclistCount'].mean().round(0).reset_index()

- (not used for 2023 base year) manual add location to `observed_bike_counts_2023.csv`

In [138]:
df_wsdot_2023.to_csv("R:/e2projects_two/2023_base_year/bike_count/wsdot_bike_apr2024.csv", index=False)
